# PyVista - Read OpenFOAM Solution Directory
- First Attempt at Reading in Entire Solution Directory

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv

## Read the OpenFOAM case file

In [2]:
case_directory = '/Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/'

In [3]:
reader = pv.POpenFOAMReader(case_directory + 'pmsCanister_v8_tas120_aoa0_900T30_part9.OpenFOAM')

## Inspect the OpenFOAM Case Boundary Conditions

In [4]:
for patch in reader.patch_array_names:
    print('Patch Name: ', patch)
print(" ")
for status in reader.all_patch_arrays_status:
    print('Patch Status: ', status)

Patch Name:  internalMesh
Patch Name:  group/freestream
Patch Name:  group/pmsCanisterGroup
Patch Name:  group/wall
Patch Name:  patch/inlet
Patch Name:  patch/outlet
Patch Name:  patch/frontAndBack
Patch Name:  patch/topAndBottom
Patch Name:  patch/pmsCanister
 
Patch Status:  internalMesh
Patch Status:  group/freestream
Patch Status:  group/pmsCanisterGroup
Patch Status:  group/wall
Patch Status:  patch/inlet
Patch Status:  patch/outlet
Patch Status:  patch/frontAndBack
Patch Status:  patch/topAndBottom
Patch Status:  patch/pmsCanister


## Read the Mesh into MultBlock object

In [5]:
mesh = reader.read()
print(f"Mesh patches: {mesh.keys()}")
internal_mesh = mesh["internalMesh"]  # or internal_mesh = mesh[0]

ERROR:root:Error reading line 22 of /Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/0/T: Expected number, string or ( for uniform entry, found Tinlet
2023-05-29 17:03:16.865 (   4.340s) [           4A814]  vtkOpenFOAMReader.cxx:8654   ERR| vtkOpenFOAMReaderPrivate (0x1060ecda0): Error reading line 22 of /Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/0/T: Expected number, string or ( for uniform entry, found Tinlet
ERROR:root:Error reading line 21 of /Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/0/U: Expected number, string or ( for uniform entry, found Uinlet
2023-05-29 17:03:16.866 (   4.342s) [           4A814]  vtkOpenFOAMReader.cxx:8654   ERR| vtkOpenFOAMReaderPrivate (0x1060ecda0): Error reading line 21 of /Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/0/U: Expected number, string or ( for uniform entry, found Uinlet
ERROR:root

Mesh patches: ['internalMesh', 'boundary']


In [6]:
mesh

MultiBlock (0x106425d60)
  N Blocks:	2
  X Bounds:	-10.000, 10.000
  Y Bounds:	-10.000, 10.000
  Z Bounds:	-10.000, 10.000

In [7]:
internal_mesh

UnstructuredGrid (0x1063d1dc0)
  N Cells:    505114
  N Points:   542768
  X Bounds:   -1.000e+01, 1.000e+01
  Y Bounds:   -1.000e+01, 1.000e+01
  Z Bounds:   -1.000e+01, 1.000e+01
  N Arrays:   6

### Inspect the OpenFOAM Case - pmsCanister

In [8]:
boundaries = mesh["boundary"]
boundaries

MultiBlock (0x139732520)
  N Blocks:	5
  X Bounds:	-10.000, 10.000
  Y Bounds:	-10.000, 10.000
  Z Bounds:	-10.000, 10.000

In [9]:
print(f"Boundaries patches: {boundaries.keys()}")

Boundaries patches: ['inlet', 'outlet', 'frontAndBack', 'topAndBottom', 'pmsCanister']


In [10]:
print(boundaries["pmsCanister"])

PolyData (0x139732820)
  N Cells:    9182
  N Points:   9257
  N Strips:   0
  X Bounds:   -3.926e-01, 3.926e-01
  Y Bounds:   -8.890e-02, 8.890e-02
  Z Bounds:   -8.885e-02, 8.885e-02
  N Arrays:   3



In [11]:
print(internal_mesh.cell_data)

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : None
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    cellLevel               float32    (505114,)
    nSurfaceLayers          float32    (505114,)
    thickness               float32    (505114,)
    thicknessFraction       float32    (505114,)


In [12]:
print(internal_mesh.point_data)

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : None
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    cellLevel               float32    (542768,)
    pointLevel              float32    (542768,)


## Inspect the final time period

In [13]:
reader.time_values

[0.0,
 50.0,
 100.0,
 150.0,
 200.0,
 250.0,
 300.0,
 350.0,
 400.0,
 450.0,
 500.0,
 550.0,
 583.0,
 600.0,
 650.0,
 700.0,
 750.0,
 800.0,
 850.0,
 900.0,
 950.0,
 1000.0]

In [14]:
reader.set_active_time_value(1000.0)
reader.cell_to_point_creation = True
mesh = reader.read()

2023-05-29 17:03:31.207 (  18.682s) [           4A814]  vtkOpenFOAMReader.cxx:9790   ERR| vtkOpenFOAMReaderPrivate (0x1060ecda0): Error reading line 20 of /Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/1000/lagrangian/kinematicCloudTracks/positions: Expected punctuation token ')', found 
ERROR:root:Error reading line 20 of /Users/jrobrien/Dissertation/data/pmsCanister/pmsCanister_v8_tas120_aoa0_900T30_part9/1000/lagrangian/kinematicCloudTracks/positions: Expected punctuation token ')', found 


In [15]:
internal_mesh = mesh["internalMesh"]
boundaries = mesh["boundary"]

## Define helper functions

In [16]:
def slice_z_center(mesh):
    """Slice mesh through center in z normal direction, move to z=0."""
    slice_mesh = mesh.slice(normal='z')
    slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
    return slice_mesh


slice_internal_mesh = slice_z_center(internal_mesh)
slice_boundaries = pv.MultiBlock({key: slice_z_center(boundaries[key]) for key in boundaries.keys()})

In [17]:
slice_boundaries

MultiBlock (0x13970fee0)
  N Blocks:	5
  X Bounds:	-10.000, 10.000
  Y Bounds:	-10.000, 10.000
  Z Bounds:	-0.000, 0.000

In [18]:
slice_internal_mesh

PolyData (0x1397321c0)
  N Cells:    3529
  N Points:   6614
  N Strips:   0
  X Bounds:   -1.000e+01, 1.000e+01
  Y Bounds:   -1.000e+01, 1.000e+01
  Z Bounds:   -4.843e-07, 4.843e-07
  N Arrays:   24

In [19]:
streamlines = slice_internal_mesh.streamlines_evenly_spaced_2D(
    vectors='U',
    start_position=(-1, 0.5, 0),
    separating_distance=1,
    separating_distance_ratio=0.1,
)

ERROR:root:vtkEvenlySpacedStreamlines2D does not support planes not aligned with XY.
2023-05-29 17:03:37.017 (  24.492s) [           4A814]vtkEvenlySpacedStreamli:123    ERR| vtkEvenlySpacedStreamlines2D (0x109d156a0): vtkEvenlySpacedStreamlines2D does not support planes not aligned with XY.
ERROR:root:Algorithm vtkEvenlySpacedStreamlines2D (0x109d156a0) returned failure for request: vtkInformation (0x6000004ef600)
2023-05-29 17:03:37.018 (  24.493s) [           4A814]       vtkExecutive.cxx:741    ERR| vtkCompositeDataPipeline (0x600001ca4500): Algorithm vtkEvenlySpacedStreamlines2D (0x109d156a0) returned failure for request: vtkInformation (0x6000004ef600)
  Debug: Off
  Modified Time: 84264
  Reference Count: 1
  Registered Events: (none)
  Request: REQUEST_DATA
  FROM_OUTPUT_PORT: 0
  ALGORITHM_AFTER_FORWARD: 1
  FORWARD_DIRECTION: 0




In [20]:
plotter = pv.Plotter()
plotter.add_mesh(slice_boundaries["pmsCanister"], color='red', line_width=3)
plotter.add_mesh(slice_boundaries["inlet"], color='black', line_width=1)
plotter.add_mesh(slice_boundaries["outlet"], color='black', line_width=1)
#plotter.add_mesh(slice_boundaries["frontAndBack"], color='black', line_width=1)
plotter.add_mesh(slice_boundaries["topAndBottom"], color='black', line_width=1)
#plotter.add_mesh(slice_boundaries["fixedWalls"], color='black', line_width=3)
#plotter.add_mesh(streamlines.tube(radius=0.0005), scalars="U")
plotter.view_xy()
#plotter.enable_parallel_projection()
plotter.show()

Widget(value="<iframe src='http://localhost:49799/index.html?ui=P_0x139735f70_0&reconnect=auto' style='width: …

In [22]:
pt = pv.Plotter()
pt.add_mesh(slice_internal_mesh)
pt.show(cpos='xy')

Widget(value="<iframe src='http://localhost:49799/index.html?ui=P_0x28ffcdeb0_2&reconnect=auto' style='width: …